In [24]:

import pandas as pd
import polars as pl
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Set seaborn style
sns.set()

# Import project configurations
from settings import config
import load_bases_data

# Retrieve directory paths from the configuration
DATA_DIR = config("DATA_DIR")
MANUAL_DATA_DIR = config("MANUAL_DATA_DIR")



# 02. Example Notebook with Dependencies

This notebook is designed demonstrate a number of goals:

  - The notebook is part of the automated analytical pipeline, as it is run programmatically by the build system, as in the dodo.py file.
  - It is tracked by version control via Git. To avoid large files and the problems associated with non-text files, the notebook is stripped of its output. 
  - In order to avoid re-running the notebook every time it changes (it changes often, even by the act of opening it) and to only rerun it if meaningful changes have been made, the build system only looks for changes in the plaintext version of the notebook. That is, the notebook is converted to a Python script via [nbconvert](https://nbconvert.readthedocs.io/en/latest/), which is often packaged with Jupyter.
  Then, DoIt looks for changes to the Python version. If it detects a difference, then the notebook is re-run. (Note, that you could also convert to a Markdown file with 
  [JupyText](https://github.com/mwouts/jupytext). However, this package is often not packaged with Jupyter.)
  - Since we want to use Jupyter Notebooks for exploratory reports, we want to keep fully-computed versions of the notebook (with the output intact). However, earlier I said that I strip the notebook of its output before committing to version control. Well, to keep the output, every time PyDoit runs the notebook, it outputs an HTML version of the freshly run notebook and saves that HTML report in the `output` directory. That way, you will be able to view the finished report at any time without having to open Jupyter.

In [25]:
from settings import config

DATA_DIR = config("DATA_DIR")  # Fetch the data directory path
ois_file = f"{DATA_DIR}/OIS.xlsx"
data_file = f"{DATA_DIR}/treasury_spot_futures.xlsx"

In [35]:
ois_file

'/Users/jameschen/FINM-32900-Final-Projcet-12-1/_data/OIS.xlsx'

In [110]:
df_ois = pd.read_excel(ois_file)


In [111]:
# 2. Identify the last business day of each month for which the contracts trade
# Read only the first column (dates) from sheet "T_SF", starting at cell A7 (skip first 6 rows)
df_dates = pd.read_excel(data_file, sheet_name="T_SF", usecols="A", skiprows=6, header=None)
df_dates.columns = ["Date"]
df_dates["Date"] = pd.to_datetime(df_dates["Date"])

In [28]:
# Compute month, year, and day
df_dates["Mat_Month"] = df_dates["Date"].dt.month
df_dates["Mat_Year"] = df_dates["Date"].dt.year
df_dates = df_dates.sort_values("Date").reset_index(drop=True)

In [29]:
# For each month, keep the last date (mimicking: keep if mofd(Date) != mofd(Date[_n+1]))
df_last = df_dates.groupby([df_dates["Date"].dt.year, df_dates["Date"].dt.month], as_index=False).agg({"Date": "last"})
df_last["Mat_Day"] = df_last["Date"].dt.day
df_last["Mat_Month"] = df_last["Date"].dt.month
df_last["Mat_Year"] = df_last["Date"].dt.year
# Drop the Date column; we need only Mat_Month, Mat_Year, Mat_Day for later merging
df_matday = df_last[["Mat_Month", "Mat_Year", "Mat_Day"]].copy()

In [30]:
# 3. Import Treasury futures data from the same Excel file and assign column names
col_names = [
    "Date", 
    "Implied_Repo_1_10", "Vol_1_10", "Contract_1_10", "Price_1_10",
    "Implied_Repo_1_5", "Vol_1_5", "Contract_1_5", "Price_1_5",
    "Implied_Repo_1_2", "Vol_1_2", "Contract_1_2", "Price_1_2",
    "Implied_Repo_1_20", "Vol_1_20", "Contract_1_20", "Price_1_20",
    "Implied_Repo_1_30", "Vol_1_30", "Contract_1_30", "Price_1_30",
    "Implied_Repo_2_10", "Vol_2_10", "Contract_2_10", "Price_2_10",
    "Implied_Repo_2_5", "Vol_2_5", "Contract_2_5", "Price_2_5",
    "Implied_Repo_2_2", "Vol_2_2", "Contract_2_2", "Price_2_2",
    "Implied_Repo_2_20", "Vol_2_20", "Contract_2_20", "Price_2_20",
    "Implied_Repo_2_30", "Vol_2_30", "Contract_2_30", "Price_2_30"
]

df = pd.read_excel(data_file, sheet_name="T_SF", skiprows=6, header=None)
df.columns = col_names

# Drop rows with missing Date (mimicking: drop if mi(Date))
df = df.dropna(subset=["Date"])
df["Date"] = pd.to_datetime(df["Date"])

In [31]:
# 4. Convert numeric columns (destring in Stata)
for col in df.columns:
    if col.startswith("Implied_Repo") or col.startswith("Vol_") or col.startswith("Price_"):
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [32]:
# 5. Reshape wide-to-long so that rows are by Date and Tenor.
#    The columns follow the pattern: <stub>_<contract>_<tenor>
stubnames = [
    "Contract_1", "Contract_2", 
    "Implied_Repo_1", "Implied_Repo_2", 
    "Vol_1", "Vol_2", 
    "Price_1", "Price_2"
]
# Use wide_to_long. The new variable "Tenor" will be extracted from the trailing digits.
df_long = pd.wide_to_long(df, stubnames=stubnames, i="Date", j="Tenor", sep="_", suffix=r'\d+')
df_long = df_long.reset_index()
# At this point, each row represents a given Date and Tenor (e.g. 10, 5, 2, 20, 30)

In [ ]:

# 6. Drop observations before June 22, 2004 (mimicking: drop if Date <= mdy(6,22,2004))
df_long = df_long[df_long["Date"] > pd.Timestamp(2004, 6, 22)].reset_index(drop=True)

# 7. Compute time-to-maturity (TTM) for each contract (nearby: v=1, deferred: v=2)
# Define a helper function for extracting maturity info from a contract string
month_map = {"DEC": 12, "MAR": 3, "JUN": 6, "SEP": 9}

def compute_ttm(contract_str, trade_date):
    if pd.isna(contract_str) or len(contract_str) < 6:
        return np.nan, np.nan  # return TTM and constructed maturity date as nan
    # Extract the three-letter month code and map to a number
    mcode = contract_str[:3]
    mat_month = month_map.get(mcode, np.nan)
    # Extract year (characters 5-6) and add 2000 (Stata: substr(Contract,5,2))
    try:
        mat_year = int(contract_str[4:6]) + 2000
    except:
        mat_year = np.nan
    # Look up the last business day from df_matday for this month/year combination
    mat_day = df_matday.loc[
        (df_matday["Mat_Month"] == mat_month) & (df_matday["Mat_Year"] == mat_year),
        "Mat_Day"
    ]
    if not mat_day.empty:
        mat_day = int(mat_day.iloc[0])
    else:
        mat_day = np.nan
    # Special cases: if contract equals "DEC 21" or "MAR 22", set day = 31
    if contract_str.strip() in ["DEC 21", "MAR 22"]:
        mat_day = 31
    # Construct the maturity date and compute TTM in days
    try:
        mat_date = pd.Timestamp(year=mat_year, month=mat_month, day=mat_day)
        ttm = (mat_date - trade_date).days
    except Exception as e:
        ttm = np.nan
        mat_date = pd.NaT
    return ttm, mat_date

# Loop over contract types v=1 and 2
for v in [1, 2]:
    contract_col = f"Contract_{v}"
    ttm_col = f"TTM_{v}"
    mat_date_col = f"Mat_Date_{v}"
    # Apply the function row-wise
    ttm_list = []
    mat_date_list = []
    for idx, row in df_long.iterrows():
        ttm, mat_date = compute_ttm(row[contract_col], row["Date"])
        ttm_list.append(ttm)
        mat_date_list.append(mat_date)
    df_long[ttm_col] = ttm_list
    df_long[mat_date_col] = mat_date_list



In [124]:
df_ois = pd.read_excel(ois_file, header=None)
df_ois

# 2) Drop the first 4 columns
df_ois = df_ois.iloc[:, 4:]
df_ois

# 1) Drop the top 3 rows
df_ois = df_ois.iloc[3:, :]
df_ois = df_ois.reset_index(drop=True)

df_ois

df_ois = df_ois.drop(index=[1, 2]).reset_index(drop=True)
df_ois

df_ois.columns = df_ois.iloc[0]  # Set first row as header
df_ois = df_ois[1:].reset_index(drop=True) 

df_ois.rename(columns={df_ois.columns[0]: "Dates"}, inplace=True)
df_ois

# 1) Create a dict that maps the old names to your desired names
rename_map = {
    "USSO1Z CMPN Curncy": "OIS_1W",
    "USSOA CMPN Curncy":  "OIS_1M",
    "USSOB CMPN Curncy":  "OIS_2M",
    "USSOC CMPN Curncy":  "OIS_3M",
    "USSOF CMPN Curncy":  "OIS_6M",
    "USSO1 CMPN Curncy":  "OIS_1Y",
    "USSO2 CMPN Curncy":  "OIS_2Y",
    "USSO3 CMPN Curncy":  "OIS_3Y",
    "USSO4 CMPN Curncy":  "OIS_4Y",
    "USSO5 CMPN Curncy":  "OIS_5Y",
    "USSO7 CMPN Curncy":  "OIS_7Y",
    "USSO10 CMPN Curncy": "OIS_10Y",
    "USSO15 CMPN Curncy": "OIS_15Y",
    "USSO20 CMPN Curncy": "OIS_20Y",
    "USSO30 CMPN Curncy": "OIS_30Y"
}

# 2) Rename the columns in-place
df_ois.rename(columns=rename_map, inplace=True)

# 3) If your 'Dates' column is actually date/time, convert it
df_ois["Dates"] = pd.to_datetime(df_ois["Dates"], errors="coerce")


df_ois


,Dates,OIS_1W,OIS_1M,OIS_2M,OIS_3M,OIS_6M,OIS_1Y,OIS_2Y,OIS_3Y,OIS_4Y,OIS_5Y,OIS_7Y,OIS_10Y,OIS_15Y,OIS_20Y,OIS_30Y
0,2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299,2024-02-23,5.3305,5.3378,5.3485,5.3465,5.2831,5.0646,4.537,4.2452,4.0833,3.9822,3.8868,3.8369,3.8346,3.7956,3.5997
6300,2024-02-26,5.3315,5.3375,5.3488,5.3487,5.2936,5.0858,4.565,4.2712,4.1068,4.0035,3.9085,3.859,3.8526,3.813,3.6202
6301,2024-02-27,5.33,5.3359,5.3487,5.3468,5.2893,5.086,4.5705,4.278,4.1151,4.0168,3.926,3.8796,3.8795,3.8416,3.6483
6302,2024-02-28,5.3295,5.3375,5.3471,5.344,5.275,5.0494,4.5177,4.2295,4.067,3.971,3.8845,3.8446,3.8485,3.8135,3.6245


In [108]:
df_long

,Date,Tenor,Contract_1,Contract_2,Implied_Repo_1,Implied_Repo_2,Vol_1,Vol_2,Price_1,Price_2,TTM_1,Mat_Date_1,TTM_2,Mat_Date_2
0,2004-06-23,10,SEP 04,DEC 04,0.34755,0.64418,430979.0,905.0,108.546875,107.312500,99.0,2004-09-30,191.0,2004-12-31
1,2004-06-24,10,SEP 04,DEC 04,0.38520,0.63771,729455.0,3605.0,108.968750,107.718750,98.0,2004-09-30,190.0,2004-12-31
2,2004-06-25,10,SEP 04,DEC 04,0.24375,0.56938,396302.0,4745.0,108.984375,107.734375,97.0,2004-09-30,189.0,2004-12-31
3,2004-06-28,10,SEP 04,DEC 04,0.05321,0.47470,677521.0,2610.0,108.265625,107.015625,94.0,2004-09-30,186.0,2004-12-31
4,2004-06-29,10,SEP 04,DEC 04,0.10706,0.53205,570751.0,2525.0,108.609375,107.375000,93.0,2004-09-30,185.0,2004-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25680,2024-02-23,30,MAR 24,JUN 24,-0.29333,3.40785,1092512.0,772555.0,125.812500,127.718750,NaN,NaT,NaN,NaT
25681,2024-02-26,30,MAR 24,JUN 24,-0.89872,3.48949,1148759.0,946552.0,125.031250,126.937500,NaN,NaT,NaN,NaT
25682,2024-02-27,30,MAR 24,JUN 24,-0.27851,3.37423,550821.0,484927.0,124.562500,126.406250,NaN,NaT,NaN,NaT
25683,2024-02-28,30,MAR 24,JUN 24,-0.65368,3.39269,81033.0,332881.0,125.218750,127.093750,NaN,NaT,NaN,NaT


In [126]:
df_ois.rename(columns={"Dates": "Date"}, inplace=True)
df_ois
# Make sure df_ois["date"] is datetime
df_ois["Date"] = pd.to_datetime(df_ois["Date"])

# 4A) Merge
df_long = df_long.merge(
    df_ois,
    on="Date",  # or "Date" if your column is capitalized
    how="left"
)


In [127]:
def piecewise_ois(ttm, row):
    """
    Given TTM in days, and a row containing OIS_1W, OIS_1M, OIS_3M, OIS_6M, OIS_1Y, ...
    Return the interpolated OIS rate.
    """
    if pd.isna(ttm):
        return np.nan
    # The breakpoints in days:
    #   0-7: use OIS_1W
    #   7-30: linear from OIS_1W -> OIS_1M
    #   30-90: linear from OIS_1M -> OIS_3M
    #   90-180: linear from OIS_3M -> OIS_6M
    #   180-360: linear from OIS_6M -> OIS_1Y
    #   >360 => just use OIS_1Y (Stata uses up to 1Y in that code)
    o1w = row["OIS_1W"]
    o1m = row["OIS_1M"]
    o3m = row["OIS_3M"]
    o6m = row["OIS_6M"]
    o1y = row["OIS_1Y"]

    t = float(ttm)
    if t <= 7:
        return o1w
    elif t <= 30:
        # linear interpolation from day=7 to day=30
        # fraction = (t - 7) / (30 - 7) = (t-7)/23
        frac = (t - 7.0) / 23.0
        return (1 - frac) * o1w + frac * o1m
    elif t <= 90:
        # from day=30 to day=90
        frac = (t - 30.0) / 60.0
        return (1 - frac) * o1m + frac * o3m
    elif t <= 180:
        # from day=90 to day=180
        frac = (t - 90.0) / 90.0
        return (1 - frac) * o3m + frac * o6m
    else:
        # from day=180 to day=360
        if t <= 360:
            frac = (t - 180.0) / 180.0
            return (1 - frac) * o6m + frac * o1y
        else:
            return o1y

for v in [1, 2]:
    df_long[f"OIS_{v}"] = df_long.apply(
        lambda row: piecewise_ois(row[f"TTM_{v}"], row),
        axis=1
    )

In [129]:
df_long["Arb_N"] = (df_long["Implied_Repo_1"] - df_long["OIS_1"]) * 100
df_long["Arb_D"] = (df_long["Implied_Repo_2"] - df_long["OIS_2"]) * 100

# We use the deferred contract
df_long["arb"] = df_long["Arb_D"]

# --- 5A) Rolling median absolute deviation, by Tenor, ±45 days ---
df_long.sort_values(["Tenor", "Date"], inplace=True)

def rolling_mad_filter(subdf, window_days=45):
    """
    For each row in subdf (all same Tenor),
    find sub-sample within ±window_days of that row’s date,
    compute median(arb), etc.
    """
    out_list = []
    for i, row in subdf.iterrows():
        d = row["Date"]
        lo = d - pd.Timedelta(days=window_days)
        hi = d + pd.Timedelta(days=window_days)
        mask = (subdf["Date"] >= lo) & (subdf["Date"] <= hi)
        subset = subdf.loc[mask, "arb"]
        med_arb = subset.median(skipna=True)
        abs_dev = abs(row["arb"] - med_arb) if pd.notna(row["arb"]) else np.nan
        out_list.append((med_arb, abs_dev))
    # Return a DataFrame with the same index
    return pd.DataFrame(out_list, columns=["arb_median", "abs_dev"], index=subdf.index)

df_list = []
for tenor, grp in df_long.groupby("Tenor"):
    grp_result = rolling_mad_filter(grp, window_days=45)
    df_list.append(pd.concat([grp, grp_result], axis=1))

df_long = pd.concat(df_list).sort_index()

# Now compute the mean of abs_dev for that rolling window, or do “median absolute dev”
# Stata does:
# rangestat (mean) mad=abs_dev, by(Tenor) interval(date -45 45) excludeself
# We can do a second pass or do it in one pass. For brevity:

def rolling_mean_abs_dev(subdf, window_days=45):
    out_list = []
    for i, row in subdf.iterrows():
        d = row["Date"]
        lo = d - pd.Timedelta(days=window_days)
        hi = d + pd.Timedelta(days=window_days)
        mask = (subdf["Date"] >= lo) & (subdf["Date"] <= hi) & (subdf.index != i)
        subset = subdf.loc[mask, "abs_dev"]
        out_list.append(subset.mean(skipna=True))
    return pd.Series(out_list, index=subdf.index)

df_list = []
for tenor, grp in df_long.groupby("Tenor"):
    grp_sorted = grp.sort_values("Date")
    grp_sorted["mad"] = rolling_mean_abs_dev(grp_sorted, 45)
    df_list.append(grp_sorted)

df_long = pd.concat(df_list).sort_values(["Tenor", "Date"])

# 5B) Mark outliers
df_long["bad_price"] = (df_long["abs_dev"] / df_long["mad"]) >= 10
df_long.loc[df_long["arb"].isna(), "bad_price"] = pd.NA  # replicate Stata's “replace bad_price = . if mi(arb)”
df_long.loc[(df_long["bad_price"] == True), "arb"] = pd.NA

# Also drop rows with no volume in the deferred contract
df_long.dropna(subset=["Vol_2"], inplace=True)


/opt/anaconda3/envs/final/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/envs/final/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/envs/final/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/envs/final/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/envs/final/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/anaconda3/envs/final/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarnin

In [132]:
df_long["T_SF_Rf"] = df_long["Implied_Repo_2"] * 100
df_long.loc[df_long["bad_price"] == True, "T_SF_Rf"] = pd.NA

df_long["tfut_ois"] = df_long["OIS_2"] * 100
df_long["T_SF_TTM"] = df_long["TTM_2"]

# Now pivot: rows => each date, columns => each Tenor
df_final = df_long.pivot(
    index="Date",
    columns="Tenor",
    values=["T_SF_Rf", "tfut_ois", "T_SF_TTM"]
)

# This yields a multi‐level column. Flatten it:
df_final.columns = [f"{col[0]}_{col[1]}" for col in df_final.columns]

# Optionally rename columns from e.g. T_SF_Rf_2 => tfut_2_rf, etc.
rename_dict = {
    "T_SF_Rf_2": "tfut_2_rf",
    "T_SF_Rf_5": "tfut_5_rf",
    "T_SF_Rf_10": "tfut_10_rf",
    "T_SF_Rf_20": "tfut_20_rf",
    "T_SF_Rf_30": "tfut_30_rf",
    "T_SF_TTM_2": "tfut_2_ttm",
    "T_SF_TTM_5": "tfut_5_ttm",
    "T_SF_TTM_10": "tfut_10_ttm",
    "T_SF_TTM_20": "tfut_20_ttm",
    "T_SF_TTM_30": "tfut_30_ttm",
    # similarly for tfut_ois_2 => ...
}

df_final.rename(columns=rename_dict, inplace=True)

df_final.reset_index(inplace=True)  # if you want 'date' as a normal column
print(df_final.head(10))

# You can then save:
df_final.to_csv(f"{DATA_DIR}/treasury_sf_implied_rf.csv", index=False)


        Date  tfut_2_rf  tfut_5_rf  tfut_10_rf  tfut_20_rf  tfut_30_rf  \
0 2004-06-23    -15.737     99.455      64.418      83.170         NaN   
1 2004-06-24    -19.993    100.954      63.771      99.725         NaN   
2 2004-06-25    -25.089     96.173      56.938      83.940         NaN   
3 2004-06-28    -16.785     97.406      47.470      70.548         NaN   
4 2004-06-29    105.306     87.968      53.205      70.995         NaN   
5 2004-06-30     92.474     71.629      28.665      13.612         NaN   
6 2004-07-01        NaN    104.109      70.184      76.897         NaN   
7 2004-07-02        NaN     89.105      67.397      88.486         NaN   
8 2004-07-06        NaN    105.806      75.098      79.200         NaN   
9 2004-07-07        NaN    104.673      78.794      88.147         NaN   

   tfut_ois_2  tfut_ois_5  tfut_ois_10  tfut_ois_20  tfut_ois_30  tfut_2_ttm  \
0  146.960000  176.817500   176.817500   176.817500          NaN        99.0   
1  147.568889  176.713889

In [133]:
url = "https://www.dropbox.com/scl/fi/81jm3dbe856i7p17rjy87/arbitrage_spread_wide.dta?rlkey=ke78u464vucmn43zt27nzkxya&st=59g2n7dt&dl=1"
df_reference = pd.read_stata(url).set_index("date")
df_reference.head()

,box_12m,box_18m,box_6m,cal_dow,cal_ndaq,cal_spx,cds_bond_hy,cds_bond_ig,cip_aud,cip_cad,...,raw_tips_treas_2,raw_tips_treas_20,raw_tips_treas_5,raw_tswap_1,raw_tswap_10,raw_tswap_2,raw_tswap_20,raw_tswap_3,raw_tswap_30,raw_tswap_5
date,,,,,,,,,,,,,,,,,,,,,
2001-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.145401,NaN,...,NaN,NaN,NaN,7.6,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.005219,NaN,...,NaN,NaN,NaN,8.1,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.061096,NaN,...,NaN,NaN,NaN,4.8,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.456711,NaN,...,NaN,NaN,NaN,5.3,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.464584,NaN,...,NaN,NaN,NaN,2.2,NaN,NaN,NaN,NaN,NaN,NaN
